In [1]:
import tensorflow as tf

tf.__version__

'2.1.0'

# 自定义梯度计算过程

In [13]:
import tensorflow as tf

class ExampleRandomNormal(tf.keras.initializers.Initializer):

    def __init__(self, weights):
        self.weights = weights

    def __call__(self, shape, dtype=None):
        return self.weights

    def get_config(self):  # To support serialization
        return {'weights': self.weights}

# y = 10 * x
layer = tf.keras.layers.Dense(1, kernel_initializer=ExampleRandomNormal([[10]]), input_shape=[None, 1])
model = tf.keras.Sequential([layer])

x = tf.Variable([[1]], dtype=tf.float64)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    # watch的监控对象必须是tf.Variable类型
    tape.watch(x)
    # J = y = 10 * x - 5，梯度应该是10
    J = model(x) - 5
gradients = tape.gradient(J, x)
print (gradients)
optimizer = tf.keras.optimizers.SGD()
# optimizer.apply_gradients([(J, x)]) # 不知道为什么报错

tf.Tensor([[10.]], shape=(1, 1), dtype=float64)


# 版本问题

2.0版本新旧对比：
https://docs.google.com/spreadsheets/d/1FLFJLzg7WNP6JHODX5q8BDgptKafq_slHpnHVbJIteQ/edit#gid=0

In [2]:
tf.enable_eager_execution
# 这句话1.x版本需要，2.x版本不需要

AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [2]:
import tensorflow as tf

tf.initialize_all_variables()
# 这句话是1.x的，2.x要换个写法

AttributeError: module 'tensorflow' has no attribute 'initialize_all_variables'

In [3]:
tf.compat.v1.global_variables_initializer()

In [4]:
tf.Session()

AttributeError: module 'tensorflow' has no attribute 'Session'

In [5]:
tf.compat.v1.Session()

# The Session graph is empty.

In [6]:
init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as session: 
    session.run(init)                            # Initializes the variables

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [11]:
tf.compat.v1.disable_eager_execution() # 保证sess.run()能够正常运行
init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as session: 
    session.run(init)  

# Can not convert a ResourceVariable into a Tensor or Operation

In [20]:
y_hat = tf.constant(36, name='y_hat')            # Define y_hat constant. Set to 36.
y = tf.constant(39, name='y')                    # Define y. Set to 39

loss = tf.Variable((y - y_hat)**2, name='loss')  # Create a variable for the loss
tf.compat.v1.disable_eager_execution()
init = tf.compat.v1.global_variables_initializer()
# init = tf.global_variables_initializer()         # When init is run later (session.run(init)),
                                                 # the loss variable will be initialized and ready to be computed
with tf.compat.v1.Session() as session: 
# with tf.Session() as session:                    # Create a session and print the output
    session.run(init)                            # Initializes the variables
    print(session.run(loss))                     # Prints the loss
# 原因：接收的参数名和run（）里面的参数名一样了，这样的话，第一次不会报错，下一次运行中，test_fc1，test_fc2变量名已有了，直接跑会和你前面定义的test_fc1，test_fc2相关运算冲突。 所以将接收的变量名改了就可以了。

9


# module 'tensorflow_core._api.v2.train' has no attribute 'AdamOptimizer'

In [22]:
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])